# First pass through the data

In [4]:
import numpy as np
import pandas as pd
import tifffile
import cv2 as cv
import os
import math
import matplotlib.pyplot as plt
import glob
import gc
import tensorflow as tf
import sys
import skimage.morphology


In [5]:
#Basic parameters 

path = r"C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney"



In [6]:
#Whats in the folder
print('\n'.join(os.listdir(path)))

.git
Code_utils
HuBMAP-20-dataset_information.csv
hubmap-kidney-segmentation.zip
README.md
sample_submission.csv
test
train
train.csv


In [7]:
train_files = sorted(glob.glob(os.path.join(path, 'train/*.tiff')))
print(f'Number of training images: {len(train_files)}')
print('\n'.join(train_files))

Number of training images: 15
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\0486052bb.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\095bf7a1f.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\1e2425f28.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\26dc41664.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\2f6ecfcdf.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\4ef6695ce.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\54f2eec69.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\8242609fa.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\aaa6a05cc.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\afa5e8098.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\b2dc8411c.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\b9a3865fc.tiff
C:\Users\victo\Desktop

In [8]:
test_files = sorted(glob.glob(os.path.join(path, 'test/*.tiff')))
print(f'Number of test images: {len(test_files)}')
print('\n'.join(test_files))

Number of test images: 5
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\test\2ec3f1bb9.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\test\3589adb90.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\test\57512b7f1.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\test\aa05346ff.tiff
C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\test\d488c759a.tiff


In [ ]:
for f in train_files:
    image = tifffile.imread(f)
    print(f'Image {f} shape: {image.shape}', flush=True)
    del image
    gc.collect()
    

In [ ]:
def rel_decoder(rle_mask, image_shape):
    split = rle_mask.split()
    start, length = split[0:][::2]
    

In [ ]:
# Utility Functions
def rle_to_image(rle_mask, image_shape):
    """
    Converts an rle string to an image represented as a numpy array.
    Reference: https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode

    :param rle_mask: string with rle mask.
    :param image_shape: (width, height) of array to return
    :return: Image as a numpy array. 1 = mask, 0 = background.
    """

    # Processing
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    image = np.zeros(image_shape[0] * image_shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        image[lo:hi] = 1

    return image.reshape(image_shape).T

In [11]:
image = tifffile.imread(r'C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\0486052bb.tiff')
image.shape

(25784, 34937, 3)

In [ ]:
class image_exploration:
    
    def __init__(self, image_file,train_df, df_info):
        self.image_file = image file
        self.train_df = train_df
        self.info = df_info
        self.glob_scale = 0.25
        
        
        
    def _image_analysis(self):
        image_id = self.get_image_id(self.image_file)
        image, image_shape = self.read_image(self.image_file)
        mask = self.read_mask(image, image_shape, self.glob_scale)
        
        HR_mask = self.read_mask(image, image_shape, scale=1)
        glom_df = self.get_particles(HR_mask, scale=1)
        glom_df['Id'] = image_id
        del HR_mask
        gc.collect()
        
        info = self.info[self.info['image_file'] == f'{image_id}.tiff']
        print(f'Image ID:        {image_id:}')
        print(f'Image Size:      {info["width_pixels"].values[0]} x {info["height_pixels"].values[0]}')
        print(f'Patient No:      {info["patient_number"].values[0]}')
        print(f'Sex:             {info["sex"].values[0]}')
        print(f'Age:             {info["age"].values[0]}')
        print(f'Race:            {info["race"].values[0]}')
        print(f'Height:          {info["height_centimeters"].values[0]} cm')
        print(f'Weight:          {info["weight_kilograms"].values[0]} kg')
        print(f'BMI:             {info["bmi_kg/m^2"].values[0]} kg/m^2')
        print(f'Laterality:      {info["laterality"].values[0]}')
        print(f'Percent Cortex:  {info["percent_cortex"].values[0]} %')
        print(f'Percent Medulla: {info["percent_medulla"].values[0]} %')
        
        
        

In [9]:
train_df = pd.read_csv(os.path.join(path,'train.csv'))

rle_mask = train_df['encoding'][0]